In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import regex
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [85]:
df = pd.read_csv('Billboard_100_dataset.csv')#,  names=['wiki_id', 'freebase_id', 'title', 'author', 'date', 'genres', 'summary'])
df = df[['Artist', 'Genre', 'Lyrics']]
df.rename(columns={'Artist': 'artist', 'Genre':'genre', 'Lyrics':'lyrics'}, inplace = True)
df.head()
# print (f'number of songs are {len(df)}')

/var/folders/wq/8n9bg9qs4yj4bzfc3wzs08f80000gn/T/ipykernel_26718/1199159047.py:1: DtypeWarning: Columns (3,5,6,22,28,102) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Billboard_100_dataset.csv')#,  names=['wiki_id', 'freebase_id', 'title', 'author', 'date', 'genres', 'summary'])


,artist,genre,lyrics
0,Gino & Gina,Rock,(It's Been A Long Time) Pretty Baby Gino & Gin...
1,Johnny Mathis,Vocal,"A certain smile, a certain face Can lead an un..."
2,Jimmy Clanton,Rock,I've been restless every night Since that nigh...
3,Everly Brothers,Rock,"[Intro] Dream, dream, dream, dream Dream, drea..."
4,Johnny Cash,Country,[Verse] Every time I look at you I fall in lov...


In [86]:
def replace_all_tokens(s):
    return re.sub(r'\[[^]?]*\]', "[VERSE]", s)

def remove_nonlatin_chars(s):
    encoded_string = s.encode("ascii", "ignore")
    decode_string = encoded_string.decode()
    return decode_string

def replace_all_missing(s):
    return re.sub(r'\[[?]\]', "[UNKNOWN]", s)
    

In [87]:
def cleaning(lyric):
    lyric = remove_nonlatin_chars(lyric)
    lyric = replace_all_tokens(lyric)
    lyric = replace_all_missing(lyric)
    return lyric

In [88]:
# remove empty lyrics
df = df.dropna()
df['clean_lyrics'] = df['lyrics'].apply(lambda x: cleaning(x))
df.head()


,artist,genre,lyrics,clean_lyrics
0,Gino & Gina,Rock,(It's Been A Long Time) Pretty Baby Gino & Gin...,(It's Been A Long Time) Pretty Baby Gino & Gin...
1,Johnny Mathis,Vocal,"A certain smile, a certain face Can lead an un...","A certain smile, a certain face Can lead an un..."
2,Jimmy Clanton,Rock,I've been restless every night Since that nigh...,I've been restless every night Since that nigh...
3,Everly Brothers,Rock,"[Intro] Dream, dream, dream, dream Dream, drea...","[VERSE] Dream, dream, dream, dream Dream, drea..."
4,Johnny Cash,Country,[Verse] Every time I look at you I fall in lov...,[VERSE] Every time I look at you I fall in lov...


In [ ]:
# save processed dataset
df.to_csv('processed_lyrics.csv', index=False)

In [3]:
train_test_ratio = 0.9
train_valid_ratio = 7/9
df_full_train, df_test = train_test_split(df, train_size = train_test_ratio, random_state = 1)
df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [4]:
def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    summaries = df['summary'].tolist()
    for summary in summaries:
        summary = str(summary).strip()
        summary = re.sub(r"\s", " ", summary)
        bos_token = '<BOS>'
        eos_token = '<EOS>'
        data += bos_token + ' ' + summary + ' ' + eos_token + '\n'
        
    f.write(data)

In [5]:
build_dataset(df_train, 'train.txt')
build_dataset(df_valid, 'valid.txt')
build_dataset(df_test, 'test.txt')